## stacking演示-波士顿房价为例
* 数据集：sklean自带的Boston房价数据集
* 问题：是一个回归问题

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,AdaBoostRegressor,ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.datasets import load_boston
from sklearn.model_selection import KFold

In [2]:
x=load_boston().data
y=load_boston().target
x=MinMaxScaler().fit_transform(x)
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=100)
kf=KFold(n_splits=5,random_state=100)
n_train=xtrain.shape[0]         #354个训练样本
n_test=xtest.shape[0]           #152个测试样本
n_train

E:\Anaconda\envs\env4SG\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


354

**第一层基模型**

In [3]:
models=[RandomForestRegressor(n_estimators=300,random_state=100),
        GradientBoostingRegressor(n_estimators=300,random_state=100),
        LGBMRegressor(n_estimators=300,n_jobs=-1,random_state=100),
        RidgeCV(alphas=[0.0001,0.001,0.01,0.1,0.2,0.5,1,2,3,4,5,10,20,30,50]),
        LinearRegression(),SVR(kernel="rbf"),SVR(kernel="linear"),
        RandomForestRegressor(n_estimators=300,random_state=100),
        GradientBoostingRegressor(n_estimators=300,random_state=100),
        XGBRegressor(n_estimators=300),
        ExtraTreesRegressor(n_estimators=300,n_jobs=-1,random_state=100)]

def get_oof(model,x_train,y_train,x_test):
    oof_train=np.zeros((n_train,))     #构造一个1*354的一维0矩阵
    oof_test=np.zeros((n_test,))       #构造一个1*152的一维0矩阵
    oof_test_skf=np.zeros((5,n_test))  #5*152
    for i,(train_index,test_index) in enumerate(kf.split(x_train)):
        kf_x_train=x_train[train_index]              #每一折训练283个样本的X
        kf_y_train=y_train[train_index]              #每一折训练283个样本的Y
        kf_x_test=x_train[test_index]                #每一折的71测试样本的X
        model=model.fit(kf_x_train,kf_y_train)
        oof_train[test_index]=model.predict(kf_x_test)        #每次产生71个预测值，最终5折后成为堆叠成为1*354个训练样本的测试值
        oof_test_skf[i,:]=model.predict(x_test)               #每次生成1*152的测试集预测值，填oof_test_skf[i，：]，五次以后填满形成5*152的预测值矩阵
    oof_test[:]=oof_test_skf.mean(axis=0)            #把测试集的五次预测结果，求平均，形成一次预测结果
    return oof_train,oof_test     #第一个返回值为第二层模型训练集的特征，1*354；第二个返回值为第一层模型对测试集数据的预测1*152，要作为第二层模型的训练集Xtest
number_models=len(models)
xtrain_new=np.zeros((n_train,number_models))
xtest_new=np.zeros((n_test,number_models))
for i,regressor in enumerate(models):
    xtrain_new[:,i],xtest_new[:,i]=get_oof(regressor,xtrain,ytrain,xtest)

**第二层模型**

In [4]:
reg=LinearRegression()
reg=reg.fit(xtrain_new,ytrain)
score=reg.score(xtest_new,ytest)
score

0.8560362640041896